In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2021-03-05 11:02:01,515 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ../../../ajna_docs/commons\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ../../../ajna_docs/commons\ajna_commons\flask\access.log


In [23]:
%load_ext autoreload 
%autoreload 2
from virasana.integracao.conformidade_alchemy import Conformidade
from virasana.scripts.conformidadeupdate import completa_conformidade, preenche_isocode, preenche_bbox
engine = create_engine(SQL_URI)
# completa_conformidade(db, engine, 200)
# preenche_isocode(db, engine, 200)
preenche_bbox(db, engine, 20)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2
2
1
2
1
2
2
1
1
1
2
1
1
1
2
1
1
1
1
2


2021-03-05 11:22:27,353 ajna         INFO     20 bbox preenchidos em 5.685836315155029 segundos.0.2842918157577515 por registro


# Checagens de conformidade

In [6]:
Session = sessionmaker(bind=engine)
session = Session()

In [7]:
rs = session.execute('SELECT count(*) FROM dbmercante.ajna_conformidade;')

In [8]:
rs.scalar()

124896

In [28]:
SQL_CONFORMIDADE = '''
SELECT cod_recinto as Recinto, count(*) as "Qtde de imagens",
 avg(height) as "Linhas",
 avg(width) as "Colunas",
 avg(ratio) as "Relação largura/altura",
 SUM(if(height < 700, 1, 0)) / count(*) * 100 as "% Tamanho pequeno",
 SUM(if(ratio < 1.9, 1, 0)) / count(ratio) * 100 as "% relação abaixo 1.9",
 avg(laplacian) as "Índice de nitidez ou ruído",
 avg(bbox_score) as "% Confiança da VC",
 sum(bbox_classe>=3) / count(*) as "% Erros da VC"
 from ajna_conformidade 
 group by cod_recinto
 '''
df = pd.read_sql(SQL_CONFORMIDADE, engine)

In [29]:
df

,Recinto,Qtde de imagens,Linhas,Colunas,Relação largura/altura,% Tamanho pequeno,% relação abaixo 1.9,Índice de nitidez ou ruído,% Confiança da VC,% Erros da VC
0,BANDEIRANTES,1725,799.3530,1925.8846,2.409826,0.0000,22.3188,113.5556,NaN,NaN
1,BTP1,27478,799.0279,2126.1514,2.660961,0.0000,4.7675,89.3529,NaN,NaN
2,BTP2,12972,799.0274,1861.6947,2.329999,0.0000,14.8551,112.0861,1.00,0.0
3,CRAGEA,640,879.9313,1912.4156,2.173406,0.0000,10.4688,NaN,NaN,NaN
4,DEICMAR,4140,799.0662,1342.7942,1.680326,0.0000,76.7874,131.3846,NaN,NaN
5,ECOPORTO,5158,798.6630,2132.7121,2.670760,0.0582,3.2959,84.3611,NaN,NaN
6,EMBRAPORT,16702,799.3164,3684.7503,4.610536,0.0000,0.1856,791.0000,0.98,0.0
7,EUDMARCO,2174,232.8270,879.4029,3.933178,93.1003,0.0000,1359.0000,NaN,NaN
8,LOCALFRIO,5402,799.0013,1760.9496,2.204347,0.0000,3.0544,112.5862,1.00,0.0
9,MARIMEX,5710,799.0285,2037.2229,2.549748,0.0000,10.5604,92.0526,1.00,0.0
